In [1]:
pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import re
import json
import requests

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Import and Concatenate Data

In [6]:
ganjar = pd.read_csv("GanjarPranowo.csv",sep=',')
prabowo = pd.read_csv("PrabowoSubianto.csv",sep=',')
anies = pd.read_csv("AniesBaswedan.csv",sep=';')

In [ ]:
# df_raw = pd.concat([ganjar,anies,prabowo], axis=0)

In [4]:
# df_raw

In [ ]:
# df_raw.info()

In [ ]:
# df_raw.to_csv("df_raw.csv")

In [ ]:
df = pd.read_csv("df_raw.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
# df['label'] = df['label'].astype('int64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6025 entries, 0 to 6024
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  6025 non-null   object
 1   label      6025 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 94.3+ KB


In [ ]:
df.head()

,full_text,label
0,Dukung Ganjar Pranowo Sebagai Capres RI 2024 #...,1
1,@ch_chotimah2 @ganjarpranowo Ganjar capres ras...,0
2,Ganjar Pranowo pemimpin yang kerja nyata untuk...,1
3,Ganjar Pranowo pemimpin yang memiliki keberani...,1
4,Ganjar Pranowo pemimpin yang visioner dan cerd...,1


In [ ]:
df.tail()

,full_text,label
6020,@Hary_Tanoe Harus memberikan kesempatan untuk ...,1
6021,"@LudmillaKarina Prabowo Gibran 🇮🇩,,",1
6022,Capres @prabowo bermusyawarah utk mufakat deng...,1
6023,"@ardi_riau5 Prabowo is the best,,",1
6024,Prabowo: Tidak ada pembicaraan soal cawapres s...,1


#Preprocessing Dataset

In [7]:
#Stopword Remover dari Sastrawi
stopword_remover = StopWordRemoverFactory().create_stop_word_remover()
#Menambahkan kamus stopword eksternal provided by DSI
list_stopword   = list(pd.read_csv('https://raw.githubusercontent.com/datascienceid/stopwords-bahasa-indonesia/master/stopwords_id_satya.txt', header = None)[0])
#menambahkan nama setiap presiden untuk dibuang dalam stopword
stopword_add = ['ganjar','pranowo','anies','baswedan','prabowo','subianto']
#menambahkan kamus slang word
response = requests.get('https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt')
slang_dictionary1 = json.loads(response.text)
#menambahkan kamus bahasa gaul/alay dari berbagai sumber luar
slang_dictionary2 = pd.read_csv('https://raw.githubusercontent.com/fendiirfan/Kamus-Alay/main/Kamus-Alay.csv')
slang_dictionary2 = slang_dictionary2.set_index('kataAlay')
slang_dictionary3 = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')
slang_dictionary3 = slang_dictionary3.filter(['slang', 'formal'], axis=1)
slang_dictionary3 = slang_dictionary3.drop_duplicates(subset=['slang'], keep='first')
slang_dictionary3 = slang_dictionary3.set_index('slang')

#Stemmer Factory dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()


def clean_text(text):
  #Case Folding : kecilkan kata menjadi huruf kecil semua
  text = text.lower()
  #Membuang kata yang mengandung http / https
  text = re.sub(r'http\S+|https\S+', '', text)
  #Membuang semua kata yang mengandung # ataupun @
  text = re.sub('(@\w+|#\w+)','',text)
  #Membuang semua non-alphabetic text
  text = re.sub('[^a-z]',' ',text)
  #Mengganti newline dengan spasi, sebab text sangat banyak newline tidak terduga
  text = re.sub("\n"," ",text)
  #Mengganti double space menjadi satu. Ini untuk merapikan teks akibat newline di atas
  text = re.sub('(s{2,})',' ',text)

  #Tokenize text
  token_text = word_tokenize(text)

  final_text = []

  for word in token_text:
    #Cek kata-kata apakah ada di kamus slang
    for key, value in slang_dictionary1.items():
      if word in key:
        word = word.replace(key, value)
        break
    if word in slang_dictionary2.index:
      word = slang_dictionary2.loc[word]['kataBaik']
    elif word in slang_dictionary3.index:
      word = slang_dictionary3.loc[word]['formal']

    #Remove stopword and Stemming by Sastrawi
    word = stopword_remover.remove(word)
    word = stemmer.stem(word)
    #Cek stopword
    if (word not in list_stopword) and type(word == str) and word != "":
      final_text.append(word)

  final_text = ' '.join(final_text)


  return final_text
  # print(final_text)




In [8]:
#Bersihkan data dari tiap presiden , untuk pembuatan Wordcloud
ganjar['clean_text'] = ganjar['full_text'].apply(clean_text)
anies['clean_text'] = anies['full_text'].apply(clean_text)
prabowo['clean_text'] = prabowo['full_text'].apply(clean_text)

#Bersihkan data total df, untuk proses modelling selanjutnya
df['clean_text'] = df['full_text'].apply(clean_text)

In [9]:
ganjar.to_csv("Ganjar_Clean.csv")
prabowo.to_csv("Prabowo_Clean.csv")
anies.to_csv("Anies_Clean.csv")

In [ ]:
#Membuang teks-teks sama dari data yang sudah diclean
duplicated_rows = df[df['clean_text'].duplicated()]
duplicated_rows.head()

,full_text,label,clean_text
11,Ganjar pemimpin yang memberi teladan bagi raky...,1,pimpin beri teladan rakyat hormat ulama
23,Ganjar Pranowo Tindak Tegas Pungli #GanjarPre...,1,tindak tegas pungli
29,Dukung Ganjar Pranowo sebagai presiden Indones...,1,dukung presiden indonesia tahun
31,Dukung Ganjar Pranowo sebagai presiden Indones...,1,dukung presiden indonesia tahun
32,Dukung Ganjar Pranowo sebagai presiden Indones...,1,dukung presiden indonesia tahun


In [ ]:
df_clean = df.drop_duplicates(subset=['clean_text'])
df_clean.reset_index(drop=True)

,full_text,label,clean_text
0,Dukung Ganjar Pranowo Sebagai Capres RI 2024 #...,1,dukung capres ri
1,@ch_chotimah2 @ganjarpranowo Ganjar capres ras...,0,capres ras mirip nazi anti israel
2,Ganjar Pranowo pemimpin yang kerja nyata untuk...,1,pimpin kerja nyata penting rakyat
3,Ganjar Pranowo pemimpin yang memiliki keberani...,1,pimpin milik berani nyali
4,Ganjar Pranowo pemimpin yang visioner dan cerd...,1,pimpin visioner cerdas
...,...,...,...
4052,@EdomorP @jokowi @gibran_tweet @prabowo Semoga...,0,moga ingat baris keluarga partai hina caci bel...
4053,@ganjarpranowo Kami nyaman bersama Prabowo-Gib...,1,nyaman gibran
4054,@Hary_Tanoe Harus memberikan kesempatan untuk ...,1,beri milenial punya punya prestasi gibran the ...
4055,Capres @prabowo bermusyawarah utk mufakat deng...,1,capres musyawarah mufakat ketua umum koalisi c...


In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4057 entries, 0 to 5023
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   full_text   4057 non-null   object
 1   label       4057 non-null   int64 
 2   clean_text  4057 non-null   object
dtypes: int64(1), object(2)
memory usage: 126.8+ KB


In [ ]:
df_clean.head()

,full_text,label,clean_text
0,Dukung Ganjar Pranowo Sebagai Capres RI 2024 #...,1,dukung capres ri
1,@ch_chotimah2 @ganjarpranowo Ganjar capres ras...,0,capres ras mirip nazi anti israel
2,Ganjar Pranowo pemimpin yang kerja nyata untuk...,1,pimpin kerja nyata penting rakyat
3,Ganjar Pranowo pemimpin yang memiliki keberani...,1,pimpin milik berani nyali
4,Ganjar Pranowo pemimpin yang visioner dan cerd...,1,pimpin visioner cerdas


In [ ]:
df_clean.to_csv("df_cleanse.csv")